# Load Model

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("shorthillsai/flan-t5-large-absa", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("shorthillsai/flan-t5-large-absa", truncation=True)

prompt = """Find the aspect based sentiment for the given review. 'Not present' if the aspect is absent.\n\nReview:I love the screen of this laptop and the battery life is amazing.\n\nAspect:Battery Life\n\nSentiment: """

input_ids = tokenizer(prompt, return_tensors="pt").to("cuda").input_ids
instruct_model_outputs = model.generate(input_ids=input_ids)
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)


c:\Users\Nimra\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


# Load Dataset

In [81]:
import os
import pandas as pd
path = "flan_t5/results/"
data_path="MAMS/"
filename="ATSA_test"
if(os.path.isfile(path+data_path+filename+"_t5.csv")):
    res = pd.read_csv(path+data_path+filename+"_t5.csv")
else:
    res = pd.DataFrame(columns=['dataset_name', 'sentence', 'aspect', 'polarity', 'sentiment_t5'])
print(len(res))
res.head(3)

1336


,dataset_name,sentence,aspect,polarity,sentiment_t5
0,MAMS/ATSA_test,The food was served promptly but the meal wasn...,food,neutral,Positive
1,MAMS/ATSA_test,The food was served promptly but the meal wasn...,served,positive,Positive
2,MAMS/ATSA_test,The food was served promptly but the meal wasn...,appetizers,positive,Positive


In [82]:
import pandas as pd

df = pd.read_csv('processed_'+data_path+filename+'.csv')
# df = pd.read_csv('processed_'+data_path+filename+'_single.csv')
print(len(df))
df.head(2)

500


,sentence,aspect_terms,polarity
0,The food was served promptly but the meal wasn...,"['food', 'served', 'appetizers']","['neutral', 'positive', 'positive']"
1,"When I got home, there was a message on the ma...","['owner', 'waitress', 'wine']","['neutral', 'negative', 'neutral']"


# Iterate each Row

In [79]:
len(res)

0

In [80]:
import ast
print(len(df), len(res))
# for sentence, aspects, sentiments in zip(df['Text'][len(res):1],df['Aspect'][len(res):1], df['Sentiment'][len(res):1]):
for sentence, aspects, sentiments in zip(df['Text'][len(res):],df['Aspect'][len(res):], df['Sentiment'][len(res):]):
    for aspec, senti in zip(aspects.split(','), sentiments.split(',')):
    # for aspec, senti in zip(ast.literal_eval(aspects), ast.literal_eval(sentiments)):
    
        temp = pd.DataFrame(columns=['dataset_name', 'sentence', 'aspect', 'polarity', 'sentiment_t5'])   
        prompt = f"""Review: {sentence}
        Aspect: {aspec.replace('#', ' ').lower()}
        Predict the sentiment for mentioned aspect in the review. Assign one of the following sentiment to the aspect: Positive , Negative , Neutral.
        Please provide the sentiment only. No any explanation or extra text"""
        print(prompt)
        input_ids = tokenizer(prompt, return_tensors="pt").to("cuda").input_ids
        instruct_model_outputs = model.generate(input_ids=input_ids)
        instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

        data_to_append = {
        'dataset_name': data_path+filename,
        'sentence': sentence,
        'aspect': aspec,
        'polarity': senti,
        'sentiment_t5': instruct_model_text_output 
        }

        print(data_to_append)
        temp = temp.append(data_to_append, ignore_index=True)   
        file_path = path+data_path+filename+"_t5.csv"
        if os.path.exists(file_path):   
            temp.to_csv(file_path, index=False, mode='a', header=False)
        else:
            temp.to_csv(file_path, index=False, mode='a')
    

106 0
Review: Too narrow . There were no rubber on the heels !
        Aspect: heels
        Predict the sentiment for mentioned aspect in the review. Assign one of the following sentiment to the aspect: Positive , Negative , Neutral.
        Please provide the sentiment only. No any explanation or extra text


c:\Users\Nimra\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'dataset_name': 'dotsa/new_Clothing_test', 'sentence': 'Too narrow . There were no rubber on the heels !', 'aspect': 'heels', 'polarity': 'Negative', 'sentiment_t5': 'Negative'}
Review: These were just too shimmery for my taste , so I returned them . But the thickness and quality is very good so if you 're going for either the Hooters looks or dance recital and need to have good coverage , these will work great .
        Aspect: thickness
        Predict the sentiment for mentioned aspect in the review. Assign one of the following sentiment to the aspect: Positive , Negative , Neutral.
        Please provide the sentiment only. No any explanation or extra text


C:\Users\Nimra\AppData\Local\Temp\ipykernel_16236\2877642716.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data_to_append, ignore_index=True)
c:\Users\Nimra\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'dataset_name': 'dotsa/new_Clothing_test', 'sentence': "These were just too shimmery for my taste , so I returned them . But the thickness and quality is very good so if you 're going for either the Hooters looks or dance recital and need to have good coverage , these will work great .", 'aspect': 'thickness', 'polarity': 'Positive', 'sentiment_t5': 'Neutral'}
Review: These were just too shimmery for my taste , so I returned them . But the thickness and quality is very good so if you 're going for either the Hooters looks or dance recital and need to have good coverage , these will work great .
        Aspect:  quality
        Predict the sentiment for mentioned aspect in the review. Assign one of the following sentiment to the aspect: Positive , Negative , Neutral.
        Please provide the sentiment only. No any explanation or extra text


...

C:\Users\Nimra\AppData\Local\Temp\ipykernel_16236\2877642716.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data_to_append, ignore_index=True)
c:\Users\Nimra\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'dataset_name': 'dotsa/new_Clothing_test', 'sentence': 'I love this watch ! I have a small wrist ( about 5 . 5 ) and larger watches never look nice . This watch is tiny and the perfect size for me . It has a few links that you can remove yourself , but I still need to go get 2 more taken out . Amazon shipped quickly , the watch is beautiful and I am very happy .', 'aspect': ' Amazon', 'polarity': ' Positive', 'sentiment_t5': 'Positive'}
Review: I love this watch ! I have a small wrist ( about 5 . 5 ) and larger watches never look nice . This watch is tiny and the perfect size for me . It has a few links that you can remove yourself , but I still need to go get 2 more taken out . Amazon shipped quickly , the watch is beautiful and I am very happy .
        Aspect:  watch--size
        Predict the sentiment for mentioned aspect in the review. Assign one of the following sentiment to the aspect: Positive , Negative , Neutral.
        Please provide the sentiment only. No any explanation 

C:\Users\Nimra\AppData\Local\Temp\ipykernel_16236\2877642716.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data_to_append, ignore_index=True)
c:\Users\Nimra\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'dataset_name': 'dotsa/new_Clothing_test', 'sentence': 'I love this watch ! I have a small wrist ( about 5 . 5 ) and larger watches never look nice . This watch is tiny and the perfect size for me . It has a few links that you can remove yourself , but I still need to go get 2 more taken out . Amazon shipped quickly , the watch is beautiful and I am very happy .', 'aspect': ' watch--size', 'polarity': ' Positive', 'sentiment_t5': 'Positive'}


C:\Users\Nimra\AppData\Local\Temp\ipykernel_16236\2877642716.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(data_to_append, ignore_index=True)


In [38]:
%pwd

'd:\\ABSA work\\codes'